In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

**Local:**

In [2]:
import constants

In [3]:
constants.pandas_settings()

In [4]:
def load_fanduel_data():
    df = pd.read_csv(f'../data/contest-files/{constants.tournament_fname()}.csv', usecols=constants.keep_cols)
    df.columns = df.columns.str.lower()
    
    # print(len(df.index))
    
    ret = (df
           .rename({'nickname': 'name'}, axis=1)
           .loc[(df['injury indicator']!='O') & (df['salary']>7000)]
           .drop('injury indicator', axis=1)
           .dropna()
           .reset_index(drop=True)
          )
    
    for c in ['played', 'salary']:
        ret[c] = ret[c].astype('int')
    
    
    
    return(ret)

In [5]:
load_fanduel_data().head(constants.display_num)

,name,fppg,played,salary
0,Rory McIlroy,88.78,13,12000
1,Scottie Scheffler,91.30,20,11900
2,Jon Rahm,80.42,14,11800
3,Justin Thomas,94.25,17,11700
4,Shane Lowry,67.93,12,11500
5,Collin Morikawa,75.92,14,11400
6,Xander Schauffele,86.26,16,11300
7,Matt Fitzpatrick,72.19,16,11200
8,Jordan Spieth,70.31,18,11100
9,Will Zalatoris,69.59,17,11000


In [6]:
strokes_gained_components = {
    'tee': {
        'url_id': 2567,
        #'url': 'https://www.pgatour.com/stats/stat.02567.html',
        'shortened': 'ott'
    },
    'approach': {
        'url_id': 2568,
        #'url': 'https://www.pgatour.com/stats/stat.02568.html',
        'shortened': 'app'
    },
    'around': {
        'url_id': 2569,
        #'url': 'https://www.pgatour.com/stats/stat.02569.html',
        'shortened': 'arg'
    },
    'green': {
        'url_id': 2564,
        #'url': 'https://www.pgatour.com/stats/stat.02564.html',
        'shortened': 'putt'
    },
    'tee-to-green': {
        'url_id': 2674,
        #'url': 'https://www.pgatour.com/stats/stat.02674.html',
        'shortened' : 'ttg'
        }
}

html_dict = {
    'tee': 'https://www.pgatour.com/stats/stat.02567.html',
    'approach': 'https://www.pgatour.com/stats/stat.02568.html',
    'around': 'https://www.pgatour.com/stats/stat.02569.html',
    'green': 'https://www.pgatour.com/stats/stat.02564.html',
    'tee-to-green': 'https://www.pgatour.com/stats/stat.02674.html'
}

new_col_names = {
    'player name': 'name',
    'rank this week': ' cur-rank',
    'rank last week': ' prev-rank',
    'average': ' sg',
    'rounds': ' num-rounds',
    'measured rounds': ' num-measured'
}

abbrev_col_names = [ 'name', ' sg' ]

In [7]:
def load_shots_gained(golf_shot, abbreviate):
    
    if golf_shot.lower() not in strokes_gained_components:
        return None
    
    else:
        info = strokes_gained_components.get(golf_shot.lower(), None)
        if info is None:
            return None
        else:
            url = f'https://www.pgatour.com/stats/stat.0{ info["url_id"] }.html'
            
            ret = pd.read_html(url)[1].reset_index(drop=True)
            
            ret.columns = ret.columns.str.lower().str.replace('total sg:', ' sg').str.replace('\xa0', ' ')
            
            ret = ret.rename(new_col_names, axis=1)
            # ret.index = ret['name']
            # ret = ret.drop('name', axis=1)
            if abbreviate:
                ret = ret.loc[:, abbrev_col_names]
                
            
            ret.columns = ret.columns.str.replace(' ', f'{strokes_gained_components[golf_shot]["shortened"]}-')
            
            
            
            return(ret)

In [8]:
# def load_shots_gained(golf_shot, abbreviate=False):
    
#     if golf_shot.lower() not in html_dict:
#         return None
    
#     else:
#         url=html_dict.get(golf_shot.lower(), None)
#         if url is None:
#             return None
#         else:
#             ret = pd.read_html(url)[1].reset_index(drop=True)
            
#             ret.columns = ret.columns.str.lower().str.replace('total sg:', ' sg').str.replace('\xa0', ' ')
            
#             ret = ret.rename(new_col_names, axis=1)
#             # ret.index = ret['name']
#             # ret = ret.drop('name', axis=1)
#             if abbreviate:
#                 ret = ret.loc[:, abbrev_col_names]
                
            
#             ret.columns = ret.columns.str.replace(' ', f'{golf_shot}-')
            
            
            
#             return(ret)

In [9]:
def compile_strokes_gained_data(abbreviate=True):
    
    sg_frames = { gs: load_shots_gained(gs, abbreviate=abbreviate) for gs in html_dict }

    sg_data = sg_frames['tee']
    for k in list(html_dict.keys())[1:]:
        sg_data = sg_data.merge(sg_frames[k])

    # sg_data['long-sg'] = sg_data['tee-sg'] + sg_data['approach-sg']
    # sg_data['short-sg'] = sg_data['green-sg'] + sg_data['around-sg']
    
    sg_data.index = sg_data['name']
    
    sg_data = (sg_data
               .sort_values(by=['putt-sg', 'arg-sg'], ascending=False)
               .drop('name', axis=1)
              )
    
    
    
    return(sg_data)

In [10]:
def combine_with_fanduel():
    fd = load_fanduel_data()
    sg = compile_strokes_gained_data()
    
    focus_stats = ( constants.focus_stat, ) if constants.focus_stat_2 is None else (constants.focus_stat, constants.focus_stat_2)
    
    for sg_col in focus_stats:
        fd[sg_col] = fd['name'].apply(lambda x: sg.loc[x, sg_col] if x in sg.index else 0.0)
        fd[f'{sg_col}-per-10k'] = np.array( 10000 * fd[sg_col] / fd['salary'] )
    
    fd = fd.sort_values(by=[f'{focus_stats[0]}-per-10k'], ascending=False)
    
    convs = {'name': 'str', 'salary': 'int'}
    
    for col in fd.columns:
        fd[col] = fd[col].astype(convs.get(col, 'float'))
    
    # fd.index = fd['name']
    # fd = fd.drop('name', axis=1)
    fd = fd.dropna()
    # print(len(fd.index))
    return(fd)

In [12]:
combine_with_fanduel().head(50)

,name,fppg,played,salary,putt-sg,putt-sg-per-10k,arg-sg,arg-sg-per-10k
41,Lucas Herbert,45.33,15.00,8900,0.80,0.90,0.20,0.23
90,Mackenzie Hughes,54.81,21.00,7700,0.61,0.79,0.31,0.41
55,Kevin Kisner,56.49,17.00,8500,0.57,0.67,0.32,0.37
17,Tyrrell Hatton,64.89,14.00,10200,0.67,0.66,0.22,0.21
36,Billy Horschel,69.45,15.00,9100,0.53,0.58,0.41,0.45
76,Ian Poulter,48.21,13.00,8000,0.44,0.55,0.24,0.30
25,Marc Leishman,66.27,18.00,9600,0.52,0.55,0.11,0.11
105,Wyndham Clark,55.13,24.00,7400,0.40,0.54,0.08,0.10
18,Sam Burns,77.47,20.00,10100,0.54,0.53,0.19,0.19
0,Rory McIlroy,88.78,13.00,12000,0.60,0.50,0.21,0.18


In [13]:
import itertools
from itertools import combinations

from functools import cache
from tqdm.notebook import tqdm

#from pandarallel import pandarallel
#pandarallel.initialize(use_memory_fs=True)

In [14]:
MIN_SALARY = 9000

In [15]:
def filters(df):
    ret = df.copy(deep=True)
    
    ret = ret.loc[ ret['salary']>=MIN_SALARY ].reset_index(drop=True)
    
    return(ret)

In [16]:
data = combine_with_fanduel() #.head(50)
data = filters(data)

pnames = data['name'].values.tolist()
data['salary'] /= 100

data.index = data['name']
data = data.drop('name', axis=1)

cols_to_sum = ['salary', constants.focus_stat, constants.focus_stat_2, f'{constants.focus_stat}-per-10k', f'{constants.focus_stat_2}-per-10k']
cost_range = range(595,601)

@cache
def get_value(name, column):
    return( data.loc[name, column] )

@cache
def sum_values(names, column):
    return( sum( [ get_value(name, column) for name in names ] ) )

@cache
def is_valid_lineup(lineup):
    return( sum_values(lineup, 'salary') in cost_range and len(set(lineup))==6 )

@cache
def lineup_analysis(lineup):
    return(tuple( [ sum_values(tuple(set(lineup)),column) for column in cols_to_sum ] ) )

def lineup_analysis_wrapper(lineup):
    return( lineup_analysis(tuple(set(lineup.to_numpy()))) if is_valid_lineup(tuple(set(lineup.to_numpy()))) else (0.0,)*len(cols_to_sum)  )

def create_lineup_2_slices(slate_dict):
#     2 things of three
    ret_list = list()
    
    for half_slates in tqdm( [p for p in itertools.product(*slate_dict.values())] ):
        
        g1,g2,g3 = tuple(sorted(list(half_slates[0])))
        g4,g5,g6 = tuple(sorted(list(half_slates[1])))
        
        lu = (g1,g2,g3,g4,g5,g6)
        if is_valid_lineup(lu):
            ret_list.append(lu)
    
    return(tuple(ret_list))

def create_lineup_3_slices(slate_dict):
    
#     3 things of two
    return None

# Trying to get better about only passing tuples or other completely immutable for default and for cache
def create_lineups(slices = (2,)):
    
    # Not necessary but makes reading easier
    num_players = 6 # (n)
    num_slices = slices[0]
    step = int( len(pnames) / num_slices )
    
    r = int(num_players / num_slices) # (nCr)
    
    # for i in range(1,num_slices+1):
        # print(int(i*step))
    
    
    # slates = {f'slate{i+1}': tuple(map( tuple, itertools.combinations(pnames[:int(i*step)], r) )) for i in range(num_slices+1)}
    # slates.update( { f'slate{num_slices}': tuple(map( tuple, itertools.combinations(pnames[int(num_slices*step):], r) )) } )
    
    slates = dict()
    
    if num_slices == 2:
        
        slates = {
            'slate1': tuple(map( tuple, itertools.combinations(pnames[:step], r) )),
            'slate2': tuple(map( tuple, itertools.combinations(pnames[step:], r) ))
        }
        
    elif num_slices == 3:
        
        slates = {
            'slate1': tuple(map( tuple, itertools.combinations(pnames[:step], r) )),
            'slate2': tuple(map( tuple, itertools.combinations(pnames[step:int(2*step)], r) )),
            'slate3': tuple(map( tuple, itertools.combinations(pnames[int(2*step):], r) )),
            
        }
    
    operations = { 2: create_lineup_2_slices(slates), 3: create_lineup_3_slices(slates)}
    
    lineups = operations[num_slices]
    
    ret = pd.DataFrame(lineups, columns=['g1','g2','g3','g4','g5','g6'])
    ret[cols_to_sum] = ret.apply( lineup_analysis_wrapper, axis=1, result_type='expand')
    #ret[cols_to_sum] = ret.parallel_apply( lineup_analysis_wrapper, axis=1, result_type='expand' )
    
    ret['avg_value'] = ret[f'{constants.focus_stat}-per-10k'] + ret[f'{constants.focus_stat_2}-per-10k']
    ret['avg_value'] /= 2
    
    ret = (ret
           # .sort_values(by=f'{constants.focus_stat}-per-10k', ascending=False)
           .sort_values(by='avg_value', ascending=False)
           .drop_duplicates()
           .reset_index(drop=True)
          )
    
    
    ret.to_csv(f'../data/lineups-created/{constants.tournament_fname()}.csv', index=False)
    
    print('Done...')
    
    return None

def output_lineups(top_num=100):
    return(pd.read_csv(f'../data/lineups-created/{constants.tournament_fname()}.csv').head(top_num))
    

In [17]:
# if constants.create:
#     create_lineups()

# output_lineups()

In [18]:
def create_func():
    if constants.create:
        create_lineups()
    return(output_lineups())

In [19]:
create_func()

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html